# Calibration transient basics

We can now try to calibrate the model over the collected data to better match the dataset.

The calibration can't be considering that the heat flow is constant since the use case is transient and not steady-state.

We then have to use the temperature at the next time step to compute how a modification of our system will affect the resulting value and iterate on that.

The steps are:
- create a model
- load the characteristics
- load the operation data
- create and run the calibration

In [5]:
from cpu.systems import CPUSystem

cpu = CPUSystem.load("data/cpu_ref.json")

In [6]:
import numpy as np
import pandas as pd

from cosapp.drivers import NonLinearSolver, EulerExplicit
from cosapp.drivers.time.scenario import Interpolator
from cosapp.recorders import DataFrameRecorder

time_driver = cpu.add_driver(EulerExplicit())
solver = time_driver.add_child(NonLinearSolver("solver", max_iter=10, factor=1.0, tol=1e-6))

# load the operation data
data = pd.read_csv("data/cpu_hot_day_intensive_use.csv")
next_T = data["T_cpu"][1:]

# define a calibration methodology
solver.add_equation("cpu.next_T == cpu.expected_next_T").add_unknown("exchanger.h_adder")

time_driver.time_interval = (0, 29)
time_driver.dt = 1.0

# define a simulation scenario using the operating conditions
time_driver.set_scenario(
    init={"T_cpu": 10.0},
    values={
        "cpu.expected_next_T": Interpolator(np.stack([np.linspace(0, 29, 30), next_T], axis=1)),
        "fan.T_air": Interpolator(np.stack([data.index, data["fan.T_air"]], axis=1)),
        "cpu.usage": Interpolator(np.stack([data.index, data["cpu.usage"]], axis=1)),
    },
)

rec = time_driver.add_recorder(DataFrameRecorder(includes=["*"], hold=False), period=1.0)

cpu.run_drivers()
rec.data.to_csv("data/calibrated_simulation.csv")

Now plot the results to understand how the exchanger conductivity adder has been computed to fit the data

In [7]:
from cpu.utils.plot_recorders import plot_recorders

plot_recorders(
    {
        "operation": pd.read_csv("data/cpu_hot_day_intensive_use.csv"),
        "simulation": pd.read_csv("data/simulation_at_operating_conditions.csv"),
        "calibrated": pd.read_csv("data/calibrated_simulation.csv"),
    },
    [
        [("time", "fan.T_air"), ("time", "cpu.usage")],
        [("time", "fan.tension"), ("time", "T_cpu")],
        [("time", "exchanger.h_adder")],
    ],
    width=800,
    height=600,
)

    'data': [{'mode': 'markers',
              'name': 'operation…

In [8]:
np.median(rec.data["exchanger.h_adder"])

np.float64(-30.328703418548095)